In [1]:
import netCDF4 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import datetime as dt
import xarray as xr
import pyproj
import requests


In [ ]:
URL = 'https://thredds.met.no/thredds/dodsC/meps25epsarchive/2017/01/01/meps_subset_2_5km_20170101T06Z.nc'
variables = ['precipitation_amount_acc', 
             'air_pressure_at_sea_level', 'air_temperature_2m', 
             'relative_humidity_2m' , 
             'air_temperature_lowest_level', 
             'x_wind_10m', 
             'y_wind_10m', 
             'integral_of_surface_downward_sensible_heat_flux_wrt_time', 
             'integral_of_surface_downwelling_longwave_flux_in_air_wrt_time', 
             'integral_of_surface_downwelling_shortwave_flux_in_air_wrt_time', 
             'integral_of_surface_net_downward_shortwave_flux_wrt_time', 
             'atmosphere_boundary_layer_thickness', 
             'high_type_cloud_area_fraction', 
             'low_type_cloud_area_fraction', 
             'medium_type_cloud_area_fraction', 
             'cloud_area_fraction', 
             'fog_area_fraction', 
             'air_temperature_pl', 
             'geopotential_pl', 
             'relative_humidity_pl' , 
             'x_wind_pl', 
             'y_wind_pl']

ds = xr.open_dataset(URL)



In [2]:
URL = 'https://thredds.met.no/thredds/dodsC/meps25epsarchive/2017/01/01/meps_subset_2_5km_20170101T06Z.nc'
ds = xr.open_dataset(URL)

variables = ['precipitation_amount_acc', 
             'air_pressure_at_sea_level', 'air_temperature_2m', 
             'relative_humidity_2m' , 
             'air_temperature_lowest_level', 
             'x_wind_10m', 
             'y_wind_10m', 
             'integral_of_surface_downward_sensible_heat_flux_wrt_time', 
             'integral_of_surface_downwelling_longwave_flux_in_air_wrt_time', 
             'integral_of_surface_downwelling_shortwave_flux_in_air_wrt_time', 
             'integral_of_surface_net_downward_shortwave_flux_wrt_time', 
             'atmosphere_boundary_layer_thickness', 
             'high_type_cloud_area_fraction', 
             'low_type_cloud_area_fraction', 
             'medium_type_cloud_area_fraction', 
             'cloud_area_fraction', 
             'fog_area_fraction', 
             'air_temperature_pl', 
             'geopotential_pl', 
             'relative_humidity_pl' , 
             'x_wind_pl', 
             'y_wind_pl']


# Compute projected coordinates of lat/lon point
lon=5.32415
lat= 60.39299
proj = pyproj.Proj(ds.projection_lambert.proj4) 
X,Y = proj(lon,lat)
dist_threshold = 3000

# Find grid nearest lon/lat point
#da = ds.sel(x=X,y=Y,method='nearest')
# Find row numer of chosen x and y coordinate (for the given lat/lon)
ix = ds.indexes['x'].get_loc(X,method='nearest')
iy = ds.indexes['y'].get_loc(Y,method='nearest')

# Select subset of points around chosen lat/lon to check distance
# Here +- 5 is chosen
check_points=ds.isel(x=slice(ix-5, ix+5), y=slice(iy-5,iy+5))
indx_x = []
indx_y = []

#calc distance for each x and y in sheck points. 
for i in range(len(check_points.x)):
    for j in range(len(check_points.y)):
        dx = abs(check_points.x[i] - X)
        dy = abs(check_points.y[j] - Y)
        distance = np.sqrt(dx**2 + dy**2)
        
        # If total distance is less than theshold, add index of point to lists indx_x, indx_y
        if distance <= dist_threshold:
            indx_x.append(ix - 5 + i)
            indx_y.append(iy -5 + j)
        else:
            continue
            
# select data from the grids within threshold distance, and take the mean over this dimension
xsel=ds.x[indx_x]
ysel=ds.y[indx_y]
sel_ds = ds.sel(x=xsel,y=ysel)

In [ ]:
mean = ds.relative_humidity_2m.mean(dim=('x','y'),skipna=True)


In [16]:
print(t2m.shape)

(67, 1, 10, 5, 5)


In [7]:
t2m = sel_ds.air_temperature_2m.values
t2m_mean = np.nanmean(t2m[:,:,:,indx_y,indx_x],axis=3)
rh2m = sel_ds.relative_humidity_2m.values
rh2m = np.nanmean(rh2m[:,:,:,indx_y,indx_x],axis=3)

dataset=xr.Dataset(data_vars={t2m:(('time','ensemble'),t2m_mean),rh2m:(('time','ensemble'),rh2m_mean)}\
                   ,coords={'time':sel_ds.time,'ensemble_member':sel_ds})
dataset.t2m.attrs["units"] = 'K'
dataset.rh2m.attrs['units'] = '%'
pathoutx='Mean.nc'
dataset.to_netcdf(path=pathoutx,mode='w')

dataset

IndexError: index 351 is out of bounds for axis 3 with size 5